In [0]:
!pip install allennlp

    100% |████████████████████████████████| 5.6MB 6.2MB/s 
    100% |████████████████████████████████| 51kB 22.2MB/s 
    100% |████████████████████████████████| 51kB 22.4MB/s 
    100% |████████████████████████████████| 245kB 18.7MB/s 
    100% |████████████████████████████████| 1.5MB 16.1MB/s 
    100% |████████████████████████████████| 133kB 22.1MB/s 
    100% |████████████████████████████████| 245kB 1.2MB/s 
    100% |████████████████████████████████| 122kB 35.0MB/s 
    100% |████████████████████████████████| 552kB 27.2MB/s 
    100% |████████████████████████████████| 51kB 21.1MB/s 
    100% |████████████████████████████████| 2.3MB 13.4MB/s 
    100% |████████████████████████████████| 143kB 32.5MB/s 
    100% |████████████████████████████████| 61kB 23.8MB/s 
    100% |████████████████████████████████| 102kB 29.4MB/s 
    100% |████████████████████████████████| 92kB 26.3MB/s 
    100% |████████████████████████████████| 9.7MB 3.1MB/s 
    100% |████████████████████████████████| 204k

In [0]:
from typing import Iterator, List, Dict
import torch
import torch.optim as optim
import numpy as np
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [0]:
torch.manual_seed(1)

In [0]:
class PosDatasetReader(DatasetReader):
    """
    DatasetReader for PoS tagging data, one sentence per line, like

        The###DET dog###NN ate###V the###DET apple###NN
    """
    def __init__(self, token_indexers: Dict[str, TokenIndexer] = None) -> None:
        super().__init__(lazy=False)
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
    def text_to_instance(self, tokens: List[Token], tags: List[str] = None) -> Instance:
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"sentence": sentence_field}

        if tags:
            label_field = SequenceLabelField(labels=tags, sequence_field=sentence_field)
            fields["labels"] = label_field

        return Instance(fields)
    def _read(self, file_path: str) -> Iterator[Instance]:
        with open(file_path) as f:
            for line in f:
                pairs = line.strip().split()
                sentence, tags = zip(*(pair.split("###") for pair in pairs))
                yield self.text_to_instance([Token(word) for word in sentence], tags)

In [0]:
class LstmTagger(Model):
    def __init__(self,
                 word_embeddings: TextFieldEmbedder,
                 encoder: Seq2SeqEncoder,
                 vocab: Vocabulary) -> None:
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                          out_features=vocab.get_vocab_size('labels'))
        self.accuracy = CategoricalAccuracy()
    def forward(self,
                sentence: Dict[str, torch.Tensor],
                labels: torch.Tensor = None) -> Dict[str, torch.Tensor]:
        mask = get_text_field_mask(sentence)
        embeddings = self.word_embeddings(sentence)
        encoder_out = self.encoder(embeddings, mask)
        tag_logits = self.hidden2tag(encoder_out)
        output = {"tag_logits": tag_logits}
        if labels is not None:
            self.accuracy(tag_logits, labels, mask)
            output["loss"] = sequence_cross_entropy_with_logits(tag_logits, labels, mask)

        return output
    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        return {"accuracy": self.accuracy.get_metric(reset)}

In [0]:
reader = PosDatasetReader()

In [0]:
train_dataset = reader.read(cached_path(
    'https://raw.githubusercontent.com/allenai/allennlp'
    '/master/tutorials/tagger/training.txt'))
validation_dataset = reader.read(cached_path(
    'https://raw.githubusercontent.com/allenai/allennlp'
    '/master/tutorials/tagger/validation.txt'))

93B [00:00, 51385.89B/s]             
2it [00:00, 3123.09it/s]
93B [00:00, 54647.00B/s]             
2it [00:00, 1340.03it/s]


In [0]:
vocab = Vocabulary.from_instances(train_dataset + validation_dataset)

100%|██████████| 4/4 [00:00<00:00, 10605.07it/s]


In [0]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

In [0]:
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})

In [0]:
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))

In [0]:
model = LstmTagger(word_embeddings, lstm, vocab)

In [0]:
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1

In [0]:
optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2, sorting_keys=[("sentence", "num_tokens")])
iterator.index_with(vocab)

In [0]:
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=300,
                  cuda_device=cuda_device)

In [0]:
trainer.train()

unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/allennlp/common/util.py", line 379, in gpu_memory_mb
    encoding='utf-8')
  File "/usr/lib/python3.6/subprocess.py", line 336, in check_output
    **kwargs).stdout
  File "/usr/lib/python3.6/subprocess.py", line 418, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.
accuracy: 0.4444, loss: 1.0370 ||: 100%|██████████| 1/1 [00:00<00:00, 86.58it/s]
accuracy: 0.4444, loss: 1.0353 ||: 100%|██████████| 1/1 [00:00<00:00, 216.51it/s]
unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/allennlp/common/util.py", line 379, in gpu_memory_mb
    encoding='utf-8')
  File "/usr/lib/python3.6/subprocess.py", line 336, in check_output
    **kwargs).stdout
  File "/usr

{'best_epoch': 299,
 'best_validation_accuracy': 1.0,
 'best_validation_loss': 0.28633707761764526,
 'epoch': 299,
 'peak_cpu_memory_MB': 448.184,
 'training_accuracy': 1.0,
 'training_cpu_memory_MB': 448.184,
 'training_duration': '00:00:13',
 'training_epochs': 299,
 'training_loss': 0.28806257247924805,
 'training_start_epoch': 0,
 'validation_accuracy': 1.0,
 'validation_loss': 0.28633707761764526}

In [0]:
predictor = SentenceTaggerPredictor(model, dataset_reader=reader)
tag_logits = predictor.predict("The dog ate the apple")['tag_logits']
tag_ids = np.argmax(tag_logits, axis=-1)

In [0]:
print([model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])

['DET', 'NN', 'V', 'DET', 'NN']


In [0]:
predictor = SentenceTaggerPredictor(model, dataset_reader=reader)
tag_logits = predictor.predict("I eat pear")['tag_logits']
tag_ids = np.argmax(tag_logits, axis=-1)
print([model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])

['NN', 'NN', 'NN']


In [0]:
with open("/tmp/model.th", 'wb') as f:
    torch.save(model.state_dict(), f)
vocab.save_to_files("/tmp/vocabulary")
# And here's how to reload the model.
vocab2 = Vocabulary.from_files("/tmp/vocabulary")
model2 = LstmTagger(word_embeddings, lstm, vocab2)
with open("/tmp/model.th", 'rb') as f:
    model2.load_state_dict(torch.load(f))
if cuda_device > -1:
    model2.cuda(cuda_device)

In [0]:
predictor2 = SentenceTaggerPredictor(model2, dataset_reader=reader)
tag_logits2 = predictor2.predict("The dog ate the apple")['tag_logits']
np.testing.assert_array_almost_equal(tag_logits2, tag_logits)

In [0]:
print(tag_logits2)

[[-0.09867547452449799, 0.7220110893249512, -0.31259220838546753], [2.0005855560302734, 0.35229408740997314, -2.3397555351257324], [-1.7287254333496094, 0.47814297676086426, 1.556679368019104], [-0.2829987406730652, 1.111972689628601, -0.3724938631057739], [2.076091766357422, 0.42547619342803955, -2.443071126937866]]
